# Temperature in Cities vs other regions in Germany
Requirements: pandas, geopandas, psycopg2 and folium (pip install ...)

In [ ]:
import folium

import pandas as pd
import geopandas as gpd

import psycopg2  # For postgres

# Connect to database
connection = psycopg2.connect(host="193.196.37.97", database="postgres",user="postgres", password="SDAPraktikum2020")
cursor = connection.cursor()

# Get all temps for stations in year 2010
#airtemps2010 = pd.read_sql_query("SELECT sts.stations_id, sts.stationsname, vals.temperature_day, vals.messdatum_date FROM air_temperature_values as vals LEFT JOIN air_temperature_stations as sts ON vals.stations_id = sts.stations_id WHERE vals.messdatum_date >= date('2010-01-01') AND vals.messdatum_date <= date('2010-12-31')", connection)
#airtemps2010

# Get all average temps for stations in year 2010
avgtemps2010 = pd.read_sql_query("SELECT sts.stationsname, AVG(vals.temperature_day) as avg_temp_yr2010 FROM air_temperature_values as vals LEFT JOIN air_temperature_stations as sts ON vals.stations_id = sts.stations_id WHERE vals.messdatum_date >= date('2010-01-01') AND vals.messdatum_date <= date('2010-12-31') GROUP BY sts.stationsname", connection)
avgtemps2010

In [ ]:
# Load station list with Landkreise political regions
stationsKreise = pd.read_csv("stations+counties.csv")
stationsKreise

# Convert to geopandas data frame with geometry column
geometry = gpd.points_from_xy(x=stationsKreise.geobreite_x, y=stationsKreise.geolaenge_x)
stationsKreise = stationsKreise.drop(['stationshoehe_x', 'geobreite_x', 'geolaenge_x'], axis=1)
stationsKreise = gpd.GeoDataFrame(stationsKreise, crs='EPSG:4326', geometry=geometry)  # "EPSG:4326" is WGS84 long-lat

# Remove columns not needed here
stationsKreise = stationsKreise.drop(['Unnamed: 0', 'von_datum', 'bis_datum'], axis=1)

stationsKreise

In [ ]:
# Join on temperatures table with statstationsname
joined = avgtemps2010.set_index('stationsname').join(stationsKreise.set_index('stationsname'))
joined

In [ ]:
# Search for rows without Kreis avialble
stationsWithoutKreis = []
for row in joined.iterrows():
    name, vals = row
    avg, kreis, geom = vals
    
    kreisname = str(kreis)
    if kreisname == "nan":
        print("No Kreis for station", name)
        stationsWithoutKreis.append(name)

In [ ]:
# Remove stations from list that have no kreis aka is NaN
kreisTemps2010 = joined.dropna()
kreisTemps2010

In [ ]:
# Create map focussed on Germany
m = folium.Map(location=[51.3, 10.1], zoom_start=6)  # This zooms the map to focus on germany (coods 51.3, 10.1)

# Add federal states overlay
germany_political = 'deutschlandGeoJSON/2_bundeslaender/2_hoch.geo.json'

folium.Choropleth(
    geo_data=germany_political,
    geo_str='choropleth',
    #data=data,
    #columns=['bundesland', 'avg_temp'],
    #key_on = 'feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Temperature (celsius)',
    highlight=True
).add_to(m)

# Add station markers from GeoPandas data frame
m.add_child(folium.features.GeoJson(stationsKreise))

# Display the map
m

In [ ]:
# Save the map output
#m.save('map.html')

In [ ]:
# Exmaple of how to render a PNG from the html map created. Only for Safari (demo), you need to set your own browser if needed

#import selenium.webdriver
#import time
#import os
#driver = selenium.webdriver.Safari()

#driver.set_window_size(1000, 800)  # choose a resolution
#path = 'file://' + os.getcwd() + '/map.html'
#path = path.replace(" ", "%20")
#driver.get(path)
#time.sleep(5)
#driver.save_screenshot('screenshot.png')